## Imports

In [9]:
import pandas as pd
import s3fs
import pandas_market_calendars as mcal

## Read Data from S3

In [10]:
s3 = s3fs.S3FileSystem(anon=True) 

In [11]:
stock_tickers = ['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW', 
                 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 
                 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT']

s3_base_path = 's3://fintech540-ml/staging/equity_data/prices/'

merged_df = pd.DataFrame()

for ticker in stock_tickers:
    file_path = f'{s3_base_path}MELTED_{ticker}_20_23.parquet.gzip'
    
    with s3.open(file_path) as f:
        df = pd.read_parquet(f)

        df.rename(columns={'Price': ticker}, inplace=True)

        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.merge(df, left_index=True, right_index=True, how='outer')


merged_df.head(3)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-01 09:30:00,317.86,228.72,95.51,145.48,119.77,173.31,47.15,90.94,117.26,38.56,...,80.22,182.49,98.40,116.0,106.9,304.02,56.70,194.73,42.67,123.40
2020-06-01 09:30:01,317.86,228.72,95.51,145.48,119.77,173.31,47.15,90.94,117.26,38.56,...,80.22,182.49,98.40,116.0,106.9,304.02,56.70,194.73,42.67,123.40
2020-06-01 09:30:02,317.86,228.72,95.40,145.51,119.77,173.31,47.15,90.98,117.26,38.52,...,80.28,182.41,98.28,116.0,106.9,304.02,56.87,194.95,42.67,123.44


## Resample and percent change

In [12]:
resampled_df = merged_df.resample('30T').mean()

resampled_df.head(3)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-01 09:30:00,319.259147,229.294031,96.142552,147.506767,119.059711,173.627903,46.817225,90.563681,117.833047,38.527000,...,80.033079,182.142186,98.457092,115.624094,106.452064,302.578939,56.507264,194.764700,43.040997,122.957003
2020-06-01 10:00:00,320.957278,227.324539,97.617694,151.052353,119.877650,174.200792,46.774172,91.210878,118.371164,38.607914,...,79.946499,182.434872,99.108328,115.750967,107.362811,301.893906,56.381878,195.285631,43.189528,122.967924
2020-06-01 10:30:00,320.721983,227.021172,97.853897,152.475632,120.374190,174.021489,46.628117,91.949754,119.004118,38.697514,...,79.712542,182.140031,99.194122,116.306925,107.612100,301.646419,56.245357,194.938736,43.309500,123.547876


In [13]:
returns_df = resampled_df.pct_change() * 100

returns_df.head(3)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-01 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 10:00:00,0.531897,-0.858937,1.534328,2.403677,0.686999,0.329952,-0.091959,0.714632,0.456677,0.210019,...,-0.108180,0.160691,0.661442,0.109729,0.855547,-0.226398,-0.221895,0.267467,0.345091,0.008883
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,...,-0.292642,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628


## Map Sectors

In [14]:
sector_map = {
    'AAPL': 'Technology', 'AMGN': 'Healthcare', 'AXP': 'Financials', 
    'BA': 'Industrials', 'CAT': 'Industrials', 'CRM': 'Technology', 
    'CSCO': 'Technology', 'CVX': 'Energy', 'DIS': 'Consumer Discretionary',
    'DOW': 'Materials', 'GS': 'Financials', 'HD': 'Consumer Discretionary',
    'HON': 'Industrials', 'IBM': 'Technology', 'INTC': 'Technology',
    'JNJ': 'Healthcare', 'JPM': 'Financials', 'KO': 'Consumer Staples',
    'MCD': 'Consumer Discretionary', 'MMM': 'Industrials', 'MRK': 'Healthcare',
    'MSFT': 'Technology', 'NKE': 'Consumer Discretionary', 'PG': 'Consumer Staples',
    'TRV': 'Financials', 'UNH': 'Healthcare', 'VZ': 'Telecommunications',
    'V': 'Financials', 'WBA': 'Consumer Staples', 'WMT': 'Consumer Staples'
}

In [15]:
sector_returns = pd.DataFrame(index=returns_df.index)

for sector in set(sector_map.values()):
    sector_stocks = [stock for stock in sector_map if sector_map[stock] == sector]
    sector_returns[sector] = returns_df[sector_stocks].mean(axis=1)

top_sector = sector_returns.idxmax(axis=1)
returns_df['top_performing_sector'] = top_sector

returns_df.head(3)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,top_performing_sector
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-01 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 10:00:00,0.531897,-0.858937,1.534328,2.403677,0.686999,0.329952,-0.091959,0.714632,0.456677,0.210019,...,0.160691,0.661442,0.109729,0.855547,-0.226398,-0.221895,0.267467,0.345091,0.008883,Industrials
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,...,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628,Energy


## Create y Target

In [16]:
returns_df['top_performing_sector_t+1'] = returns_df['top_performing_sector'].shift(-1)

returns_df.head(3)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,top_performing_sector,top_performing_sector_t+1
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-01 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industrials
2020-06-01 10:00:00,0.531897,-0.858937,1.534328,2.403677,0.686999,0.329952,-0.091959,0.714632,0.456677,0.210019,...,0.661442,0.109729,0.855547,-0.226398,-0.221895,0.267467,0.345091,0.008883,Industrials,Energy
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,...,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628,Energy,Consumer Staples


In [17]:
returns_df = returns_df.drop(columns=['top_performing_sector'])
returns_df = returns_df.dropna()

returns_df.head(3)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,top_performing_sector_t+1
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:00:00,0.531897,-0.858937,1.534328,2.403677,0.686999,0.329952,-0.091959,0.714632,0.456677,0.210019,...,0.160691,0.661442,0.109729,0.855547,-0.226398,-0.221895,0.267467,0.345091,0.008883,Energy
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,...,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628,Consumer Staples
2020-06-01 11:00:00,-0.119307,-0.257543,-0.191428,-0.695396,-0.127898,-0.250637,-0.600353,-0.322545,0.028758,-0.158995,...,-0.074866,-0.177802,0.250451,-0.007171,-0.064217,-0.227855,-0.220650,-0.450690,0.258141,Healthcare


## Join Gas Data

In [18]:
gas_df = pd.read_csv('s3://fintech540-ml/external/weekly-gasoline.csv')
gas_df.columns = ['gas_' + x.lower().replace(' ', '_') for x in gas_df.columns]

In [19]:
gas_df = gas_df.sort_values(['gas_fiscal_year', 'gas_fiscal_week'])

In [20]:
returns_df = returns_df.reset_index()
returns_df['Timestamp'] = pd.to_datetime(returns_df['Timestamp'])

returns_df.head(3)

,Timestamp,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,top_performing_sector_t+1
0,2020-06-01 10:00:00,0.531897,-0.858937,1.534328,2.403677,0.686999,0.329952,-0.091959,0.714632,0.456677,...,0.160691,0.661442,0.109729,0.855547,-0.226398,-0.221895,0.267467,0.345091,0.008883,Energy
1,2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,...,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628,Consumer Staples
2,2020-06-01 11:00:00,-0.119307,-0.257543,-0.191428,-0.695396,-0.127898,-0.250637,-0.600353,-0.322545,0.028758,...,-0.074866,-0.177802,0.250451,-0.007171,-0.064217,-0.227855,-0.220650,-0.450690,0.258141,Healthcare


In [21]:
def year_week_to_timestamp(year, week):
    return pd.to_datetime(f'{year} {week} 1', format='%Y %W %w')

# Apply this function to gas dataframe
gas_df['Timestamp'] = gas_df.apply(lambda row: year_week_to_timestamp(row['gas_fiscal_year'], row['gas_fiscal_week']), axis=1)

# Sort both dataframes by Timestamp
returns_df = returns_df.sort_values(by='Timestamp')
gas_df = gas_df.sort_values(by='Timestamp')

# Use merge_asof to merge the dataframes
returns_gas_df = pd.merge_asof(returns_df, gas_df, on='Timestamp', direction='forward')

# Display the merged DataFrame
returns_gas_df.head(3)

,Timestamp,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,WBA,WMT,top_performing_sector_t+1,gas_fiscal_year,gas_fiscal_week,gas_current_year_production,gas_previous_year_production,gas_difference_from_same_week_last_year,gas_current_year_cumulative_production,gas_cumulative_difference
0,2020-06-01 10:00:00,0.531897,-0.858937,1.534328,2.403677,0.686999,0.329952,-0.091959,0.714632,0.456677,...,0.345091,0.008883,Energy,2020.0,23.0,2.778006e+09,2.687160e+09,90846000,6.145423e+10,280770000
1,2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,...,0.277781,0.471628,Consumer Staples,2020.0,23.0,2.778006e+09,2.687160e+09,90846000,6.145423e+10,280770000
2,2020-06-01 11:00:00,-0.119307,-0.257543,-0.191428,-0.695396,-0.127898,-0.250637,-0.600353,-0.322545,0.028758,...,-0.450690,0.258141,Healthcare,2020.0,23.0,2.778006e+09,2.687160e+09,90846000,6.145423e+10,280770000


## Join Layoffs Data

In [22]:
industry_layoffs = pd.read_csv('s3://fintech540-ml/staging/transform/INDUSTRY_LAYOFFS_DAILY.csv')

industry_layoffs = industry_layoffs.rename(columns={'date': 'Timestamp'})

industry_layoffs['Timestamp'] = pd.to_datetime(industry_layoffs['Timestamp'])

In [23]:
industry_layoffs = industry_layoffs.sort_values('Timestamp')

In [24]:
returns_ind_gas_df = pd.merge_asof(returns_gas_df, industry_layoffs, on='Timestamp', direction='forward')

## Filter Time to 9:30 to 4

In [27]:
returns_ind_gas_df.set_index('Timestamp', inplace=True)

returns_ind_gas_df = returns_ind_gas_df.between_time('09:30', '16:00')

returns_ind_gas_df.reset_index(inplace=True)

print(returns_ind_gas_df.head(3))

            Timestamp      AAPL      AMGN       AXP        BA       CAT  \
0 2020-06-01 10:00:00  0.531897 -0.858937  1.534328  2.403677  0.686999   
1 2020-06-01 10:30:00 -0.073310 -0.133451  0.241968  0.942242  0.414206   
2 2020-06-01 11:00:00 -0.119307 -0.257543 -0.191428 -0.695396 -0.127898   

        CRM      CSCO       CVX       DIS  ...  other_laid_off_count_sum  \
0  0.329952 -0.091959  0.714632  0.456677  ...                       0.0   
1 -0.102929 -0.312257  0.810075  0.534720  ...                       0.0   
2 -0.250637 -0.600353 -0.322545  0.028758  ...                       0.0   

   product_laid_off_count_sum  real estate_laid_off_count_sum  \
0                         0.0                             0.0   
1                         0.0                             0.0   
2                         0.0                             0.0   

   recruiting_laid_off_count_sum  retail_laid_off_count_sum  \
0                            0.0                        0.0   
1      

## WRITE TO CSV

In [31]:
cols = [x for x in returns_ind_gas_df if x != "top_performing_sector_t+1"]
final = returns_ind_gas_df[cols + ['top_performing_sector_t+1']]
final.to_csv('RETURNS_GAS_IND_TARGET.csv', index=False)